In [1]:
import cProfile
import random as rd
import time
from collections import defaultdict
import numpy as np

#import Games.othello as oth
import Games.mnk as mnk
import Agents.random as arand
#import carcassonne as carc

In [6]:
def random_game(state, agent):
    gs = state.duplicate()
    while not gs.is_terminal:
        action = agent.choose_action(gs)
        gs.make_action(action)
    return gs

base_gs = mnk.GameState(13,13,5)
base_gs.set_initial_state()
agent = arand.RandomPlayer()

In [89]:
action = agent.choose_action(base_gs)
base_gs.make_action(action)
base_gs.view_game_state()
print(str(base_gs))

[['X' 'O' 'O' ' ' 'X' 'X' 'O' 'X' ' ' ' ' 'O' 'X' ' ']
 ['O' 'O' 'X' 'O' 'O' ' ' 'X' ' ' 'O' ' ' ' ' ' ' 'O']
 [' ' 'X' ' ' 'X' ' ' ' ' 'O' 'O' 'X' ' ' 'O' ' ' ' ']
 [' ' ' ' 'O' ' ' ' ' 'X' 'O' 'X' ' ' 'O' ' ' 'X' 'X']
 ['X' 'X' 'O' ' ' 'O' 'O' ' ' 'O' 'X' 'X' 'X' ' ' 'X']
 [' ' 'O' ' ' ' ' ' ' ' ' 'O' 'X' ' ' ' ' ' ' 'O' ' ']
 ['X' ' ' ' ' 'O' 'X' 'O' ' ' ' ' 'X' 'O' 'X' ' ' 'X']
 [' ' 'X' 'X' 'X' 'O' 'O' 'O' ' ' ' ' 'X' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' 'X' ' ' 'X' ' ' 'X' 'X' 'O' 'O']
 [' ' ' ' 'O' ' ' ' ' 'O' 'O' 'O' ' ' ' ' ' ' 'X' ' ']
 [' ' ' ' ' ' ' ' ' ' 'X' ' ' ' ' 'X' ' ' 'O' 'O' ' ']
 [' ' ' ' 'X' 'O' ' ' ' ' 'O' 'O' 'X' ' ' ' ' ' ' 'X']
 [' ' ' ' ' ' ' ' 'X' ' ' ' ' ' ' ' ' ' ' ' ' 'O' 'X']]
m13n13k5terminalTruewinner0player_turn1available_actions86


In [3]:
base_gs = mnk.GameState(13,13,5)
base_gs.set_initial_state()
start = time.time()
games = 0
agent = arand.RandomPlayer()
wins = defaultdict(lambda: 0)

def random_games(games, state):
    wins = defaultdict(lambda:0)
    for _ in range(games):
        gs = state.duplicate()
        while not gs.is_terminal:
            action = agent.choose_action(gs)
            gs.make_action(action)
        wins[gs.winner] += 1
        #gs.view_game_state()
    return wins

def random_game(state):
    gs = state.duplicate()
    while not gs.is_terminal:
        action = agent.choose_action(gs)
        gs.make_action(action)
    return gs


wins =  random_games(10000, base_gs)
print(str(wins))

defaultdict(<function random_games.<locals>.<lambda> at 0x0000026841DFD6C0>, {0: 5138, 1: 4862})


In [4]:
cProfile.run("wins =  random_games(1000, base_gs)")

         845332 function calls in 0.750 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.053    0.053    0.750    0.750 2927661125.py:8(random_games)
        2    0.000    0.000    0.000    0.000 2927661125.py:9(<lambda>)
        1    0.000    0.000    0.750    0.750 <string>:1(<module>)
    88338    0.353    0.000    0.362    0.000 mnk.py:104(make_action)
     1000    0.004    0.000    0.030    0.000 mnk.py:139(duplicate)
     1000    0.011    0.000    0.011    0.000 mnk.py:143(<dictcomp>)
     1000    0.014    0.000    0.014    0.000 mnk.py:144(<dictcomp>)
     1000    0.001    0.000    0.001    0.000 mnk.py:39(__init__)
    88338    0.059    0.000    0.082    0.000 random.py:239(_randbelow_with_getrandbits)
    88338    0.045    0.000    0.136    0.000 random.py:375(choice)
    88338    0.157    0.000    0.304    0.000 random.py:7(choose_action)
        1    0.000    0.000    0.750    0.750 {built-in method

In [5]:
wins =  random_games(1000, base_gs)
print(wins)

defaultdict(<function random_games.<locals>.<lambda> at 0x000001AF1C4F5820>, {0: 491, 1: 509})


In [4]:
base_gs = mnk.GameState(5,5,3)
base_gs.set_initial_state()
agent = ag.RandomPlayer()
gs = base_gs.duplicate()

In [8]:

action = agent.choose_action(gs)
gs.make_action(action)
horizontal = gs.board[action.x]
vertical = gs.board[:,action.y]
offset = action.y-action.x
antioffset = gs.n - 1 - action.y - action.x
diagonal_up = gs.board.diagonal(offset)
diagonal_down = np.fliplr(gs.board).diagonal(antioffset)


gs.view_game_state()
print("horizontal", horizontal)
print("vertical", vertical.transpose())
print("action.x", action.x)
print("action.y", action.y)
print("offset", offset)
print("diagonal_up", diagonal_up)
print("diagonal_down", diagonal_down)

[[' ' ' ' ' ' ' ' ' ']
 ['O' 'X' ' ' ' ' ' ']
 [' ' ' ' 'X' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ']
 [' ' 'O' ' ' ' ' ' ']]
horizontal [[2 1 2 2 2]]
vertical [[2 0 2 2 1]]
action.x 4
action.y 1
offset -3
diagonal_up [[2 1]]
diagonal_down [[2 2 2 1]]
